In [60]:
def loadaccgyro(exp, user, start, end, accelerometer_x, accelerometer_y,
                accelerometer_z, gyrosensor_x, gyrosensor_y, gyrosensor_z):

    
    # 가속도 데이터 로딩
    filename = 'acc_exp' + exp + '_user' + user + '.txt'
    acc_data = np.loadtxt(filename)
    for i in range(start-1, end ):
        accelerometer_x.append(acc_data[i][0])
        accelerometer_y.append(acc_data[i][1])
        accelerometer_z.append(acc_data[i][2])

    # 자이로 데이터 로딩
    filename = 'gyro_exp' + exp + '_user' + user + '.txt'
    gyro_data = np.loadtxt(filename)
    for i in range(start-1, end):
        gyrosensor_x.append(gyro_data[i][0])
        gyrosensor_y.append(gyro_data[i][1])
        gyrosensor_z.append(gyro_data[i][2])


In [89]:
import pickle
import numpy as np

#모델 로드
with open('svm_rbf.pkl', 'rb') as f:
    model = pickle.load(f)
    
print(model)

lst = [['01', 1233, 1392], ['01', 2195, 2359], ['01', 4539, 4735], ['01', 5668, 5859],
       ['01', 6787, 6977], ['01', 6787, 6977], ['01', 6787, 6977], ['01', 6787, 6977],
       ['01', 1233, 1392], ['01', 2195, 2359], ['01', 4539, 4735], ['01', 5668, 5859]]

accelerometer_x = []
accelerometer_y = []
accelerometer_z = []
gyrosensor_x = []
gyrosensor_y = []
gyrosensor_z = []

# =====================================================================================    
# for i in range(len(lst)):
# loadaccgyro('01', lst[i][0],lst[i][1],lst[i][2], accelerometer_x, accelerometer_y,
#                    accelerometer_z, gyrosensor_x, gyrosensor_y, gyrosensor_z)
# =====================================================================================    
i = 0
exp = '01'
user = lst[i][0]
start = lst[i][1] - 1
end = lst[i][2]
length = 128
    
# 가속도 데이터 로딩
accfilename = 'acc_exp' + exp + '_user' + user + '.txt'
gyrofilename = 'gyro_exp' + exp + '_user' + user + '.txt'

acc_data = np.loadtxt(accfilename)
gyro_data = np.loadtxt(gyrofilename)

tstart = start
tend = start + 128
xtrain = []
while True:
    temp = []
    for i in range(tstart, tend):
        temp.append(acc_data[i][0])
        temp.append(acc_data[i][1])
        temp.append(acc_data[i][2])
        temp.append(gyro_data[i][0])
        temp.append(gyro_data[i][1])
        temp.append(gyro_data[i][2])

    xtrain.append(temp)
    tstart = tstart + 1
    tend = tstart + 128
    if (end < tend) :
        break

print(xtrain)


xdata = []
ydata = []   
xdata.extend(xtrain)
ydata.extend(xtrain)

xtrain = np.array(xtrain)
print(xtrain.shape)
X_tr_flat = xtrain.reshape((xtrain.shape[0],-1))
print(X_tr_flat.shape)
y_pred = model.predict(X_tr_flat)
print(y_pred)

SVC(random_state=0)
[[1.015277827209008, -0.1416666681928075, 0.05972222334229981, 0.03145955875515938, 0.003359758760780096, 0.01496619824320078, 1.020833394742025, -0.1638888897010154, 0.05555555984850197, 0.07299839705228806, 0.03390302136540413, -0.003054326167330146, 1.027777854158295, -0.1736111207278297, 0.06944444829113189, 0.1038470938801765, 0.06414084881544113, 0.005192354787141085, 1.025000070391787, -0.1638888897010154, 0.05833333753702795, 0.1038470938801765, 0.04520402848720551, 0.01282817032188177, 1.022222286625279, -0.1472222235698595, 0.04861111258819587, 0.1301143020391464, 0.03329215571284294, 0.01282817032188177, 1.022222286625279, -0.1597222262072175, 0.0430555572111439, 0.1927279829978943, 0.01374446786940098, 0.01496619824320078, 1.027777854158295, -0.1625000099737258, 0.04861111258819587, 0.2266310006380081, 0.05925392732024193, 0.007635815534740686, 1.027777854158295, -0.1652777815842695, 0.04861111258819587, 0.2822197377681732, 0.04886921867728233, 0.0042760

#### exp 01 user 01
#### 2(1233, 1392), 4(2195, 2359), 6(4539, 4735),8(5668, 5859) 번째 줄 정상 
#### 12(6787, 6977)번째 줄 비정상

### xdata로드

In [49]:
response = {"accX" : accelerometer_x,
    "accY" : accelerometer_y,
    "accZ" : accelerometer_z, 
    "gyroX" : gyrosensor_x,
    "gyroY" : gyrosensor_y,
    "gyroZ" : gyrosensor_z}

In [48]:
def create_windows(user, start, end, length, overlap):
    
    # SQS데이터 가져오기
    acc_data  = np.array([response["accX"],response["accY"], response["accZ"]])
    gyro_data = np.loadtxt([response["gyroX"], response["gyroY"], response["gyroZ"]])
    
    x_sqs = []
    
    while (start + length <= end) : 
          
        stop = start + length
        window = []
        
        while start != stop :
            window.append( [acc_data[start] , gyro_data[start]] )
            start += 1
        
        x_sqs.append(window)
        
        start = stop - overlap[activity-1]

    return x_sqs

In [37]:
def prepare_data(length = 128,overlap = [64]*12): 
    x_new = []
         
    xtemp = create_windows(user, start, end+1, length, overlap)
    x_new.extend(xtemp)
        
    return x_new

In [38]:
prepare_data(response,length = 128,overlap = [64]*12)

array([[ 1.01527783,  1.02083339,  1.02777785, ...,  0.30833334,
         0.31111113,  0.31805556],
       [-0.14166667, -0.16388889, -0.17361112, ...,  0.7527778 ,
         0.73888893,  0.74583334],
       [ 0.05972222,  0.05555556,  0.06944445, ...,  0.53611115,
         0.53750004,  0.54166667],
       [ 0.03145956,  0.0729984 ,  0.10384709, ..., -0.0271835 ,
         0.0290161 ,  0.0552833 ],
       [ 0.00335976,  0.03390302,  0.06414085, ...,  0.01527163,
         0.02840523,  0.05283985],
       [ 0.0149662 , -0.00305433,  0.00519235, ..., -0.00458149,
        -0.00488692, -0.02657264]])

In [40]:
X_new.shape

(6, 2192)

In [19]:
# 전처리

def prepare_data(location,length = 128,overlap = [64]*12):
    response = {"accX" : accelerometer_x,
        "accY" : accelerometer_y,
        "accZ" : accelerometer_z, 
        "gyroX" : gyrosensor_x,
        "gyroY" : gyrosensor_y,
        "gyroZ" : gyrosensor_z}
    
    xdata = []
    xtemp , ytemp = create_windows(location, exp, user, start, end+1, activity, length, overlap)
    xdata.extend(xtemp)
    X_new = np.array([response["accX"], response["accY"], response["accZ"], response["gyroX"], response["gyroY"], response["gyroZ"]])

        
    return X_new

 def get_scaler(X_new):

        row = X_new.shape[0]
        timestamp = X_new.shape[1]
        sensor = X_new.shape[2]
        axis = X_new.shape[3]

        scaler = MinMaxScaler(feature_range = (-1,1))
        xdata = xdata.reshape(row,timestamp,sensor*axis)
        xdata = np.swapaxes(xdata,0,2).reshape(sensor*axis,-1).T
        scaler.fit(xdata)
        return scaler
    
def scale_data(xdata,scaler):

        row = xdata.shape[0]
        timestamp = xdata.shape[1]
        sensor = xdata.shape[2]
        axis = xdata.shape[3]

        xdata = xdata.reshape(row,timestamp,sensor*axis)
        xdata = np.swapaxes(xdata,0,2).reshape(sensor*axis,-1).T
        xdata = scaler.transform(xdata)
        xdata = xdata.T.reshape(sensor*axis,timestamp,row)
        xdata = np.swapaxes(xdata,0,2).reshape(row,timestamp,sensor,axis)

        return xdata  
    
def preprocess_data(xdata):

    return 
print(preprocess_data(X_new))

None


In [17]:
# 모델에 새로운 데이터 넣고 예측
y_pred = model.predict(X_new_processed)
print(y_pred)

ValueError: Expected 2D array, got scalar array instead:
array=nan.
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.